<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/keras_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Layer

In [ ]:
from tensorflow.keras.layers import Dense, Layer, BatchNormalization, Dropout, concatenate
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.utils import plot_model
import tensorflow.keras as keras

import numpy as np

In [ ]:
dense = Dense(10)

In [ ]:
bn = BatchNormalization()

In [ ]:
isinstance(dense, Layer)

In [ ]:
# importante! settando questo attributo a False i suoi pesi non vengono aggiornati col gradient descent
dense.trainable

In [ ]:
dense.variables

In [ ]:
dense.name

In [ ]:
dense.input

In [ ]:
dense.output

In [ ]:
# inizializzazione dei pesi tramite build

In [ ]:
dense = Dense(10)

In [ ]:
dense.build((100,))

In [ ]:
dense.trainable_variables

In [ ]:
# settare l'inizializzazione

In [ ]:
dense = Dense(10, kernel_initializer='glorot_normal')

In [ ]:
dense = Dense(10, kernel_initializer=keras.initializers.GlorotNormal(seed=1))

In [ ]:
# inizializzazione mediante call

In [ ]:
dense = Dense(10)

In [ ]:
dense(np.random.rand(10, 45))

In [ ]:
bn(np.random.rand(10, 45))

In [ ]:
bn.weights

In [ ]:
bn.trainable_weights

In [ ]:
bn.non_trainable_weights

# Modelli

## Sequenziale

In [ ]:
# Posso specificare un input con il parametro input_shape del primo layer
# oppure includendo all'inizio un oggetto di tipo Input

model = Sequential(
    [
     Input(10),
     Dense(10, activation='relu', name='dense_1'),
     Dropout(0.1, name='dropout_1'),
     Dense(10, activation='relu', name='dense_2'),
     Dropout(0.1, name='dropout_2'),
     Dense(1, name='output')
    ]
)

In [ ]:
isinstance(model, Model)

In [ ]:
isinstance(model, Layer)

In [ ]:
model.layers

In [ ]:
dense_1 = model.get_layer(name='dense_1')

In [ ]:
dense_1.output

In [ ]:
dense_1.input

In [ ]:
model.summary()

In [ ]:
plot_model(model=model, show_shapes=True)

## Funzionale



In [ ]:
input_1 = Input(shape=(10,))
input_2 = Input(shape=(20,))

# prima sottorete
x = Dense(100, activation='relu')(input_1)
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
# seconda sottorete
y = Dense(100, activation='relu')(input_2)
y = Dense(100, activation='relu')(y)
y = Dense(100, activation='relu')(y)

conc = concatenate([x, y])
out = Dense(1)(conc)

model = Model(inputs=[input_1, input_2], outputs=out)

In [ ]:
model.summary()

In [ ]:
plot_model(model)

## Subclassing

In [45]:
class RegressorMLP(Model):
    def __init__(self, num_hidden_layers, activation, units_x_layer, dropout_rate):
        super().__init__(name='RegressorMLP')
        self.hidden_layers = []
        self.dropouts = []
        for i in range(num_hidden_layers):
            self.hidden_layers.append(Dense(units=units_x_layer, activation=activation, name=f'dense_{i}'))
            self.dropouts.append(Dropout(dropout_rate, name=f'dropout_{i}'))
        self.last_layer = Dense(1, name=f'dense_{num_hidden_layers}')

    def call(self, inputs, training=False):
        x = inputs
        for dense, dropout in zip(self.hidden_layers, self.dropouts):
            print(dense.name, dropout.name)
            x = dense(x)
            x = dropout(x) if training else x

        x = self.last_layer(x)
        
        return x

In [46]:
model = RegressorMLP(num_hidden_layers=4, activation='relu', units_x_layer=50, dropout_rate=0.3)

In [47]:
model(np.random.rand(32, 10).astype(np.float32), training=True)

dense_0 dropout_0
dense_1 dropout_1
dense_2 dropout_2
dense_3 dropout_3


<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[ 0.10910045],
       [ 0.45822078],
       [ 0.09547866],
       [ 0.05371245],
       [ 0.32558095],
       [ 0.10365595],
       [ 0.29161   ],
       [ 0.38242865],
       [ 0.06606817],
       [ 0.07444821],
       [ 0.18797219],
       [ 0.10027465],
       [-0.00575983],
       [ 0.20321989],
       [ 0.8152238 ],
       [ 0.24955243],
       [ 0.2227236 ],
       [-0.02099437],
       [ 0.14389856],
       [ 0.15358502],
       [ 0.38101864],
       [ 0.12439577],
       [ 0.04295134],
       [ 0.03579491],
       [ 0.19433002],
       [ 0.22707747],
       [ 0.27583084],
       [ 0.4445622 ],
       [ 0.12045928],
       [-0.13732302],
       [ 0.34698936],
       [ 0.13915986]], dtype=float32)>

# Addestramento

In [ ]:
# come si 